In [ ]:
#@title Vars

variable_groups = {  # T2 Variables
# *****Bio or Objective Child

        'Residential Characteristics': ['neighborhood_safety_ss_p', 'neighborhood_safe_y',  'resid_density', 'resid_walkability', 'resid_prox_roads', 'resid_crime_tot', 'resid_crime_violent',
            'resid_crime_drug', 'resid_crime_dui', 'resid_lead_risk_poverty', 'resid_lead_risk_houses_perc',
            'resid_lead_risk', 'resid_no2_avg', 'resid_pm25_avg', 'resid_sexism', 'resid_sex_orient_bias',
            'resid_immigrant_bias', 'resid_racism'],# 'L_site_id'],


        'Ethnicity/Nationality': ['desc_african_AFR_B', 'desc_native_american_AMR_B', 'desc_alaska_native_AMR_B',
            'desc_chinese_EAS_B', 'desc_japanese_EAS_B', 'desc_korean_EAS_B', 'desc_vietnamese_EAS_B',
            'desc_european_EUR_B', 'desc_asian_indian_SAS_B', 'desc_other_south_asian_SAS_B', 'desc_latin_B'
            'pc_gene_aces1', 'pc_gene_aces2', 'pc_gene_aces3', 'pc_gene_aces4',
            'pc_gene_aces5', 'pc_gene_aces6', 'pc_gene_aces7', 'pc_gene_aces8',
            'pc_gene_aces9', 'pc_gene_aces10', 'pc_gene_aces11', 'pc_gene_aces12',
            'pc_gene_aces13', 'pc_gene_aces14', 'pc_gene_aces15', 'pc_gene_aces16',
            'pc_gene_aces17', 'pc_gene_aces18', 'pc_gene_aces19', 'pc_gene_aces20',
            'pc_gene_aces21', 'pc_gene_aces22', 'pc_gene_aces23', 'pc_gene_aces24',
            'pc_gene_aces25', 'pc_gene_aces26', 'pc_gene_aces27', 'pc_gene_aces28',
            'pc_gene_aces29', 'pc_gene_aces30', 'pc_gene_aces31', 'pc_gene_aces32'],


        'Diet/Nutrition': ['fruit_intake', 'vegetable_intake', 'protein_sources_intake', 'legume_intake',
            'added_sugar', 'sugary_beverage_freq', 'dairy_intake', 'whole_grain_intake', 'total_calories',
            'protein_intake', 'carbohydrate_intake', 'fiber_intake', 'sodium_intake', 'potassium_intake',
            'total_sugar', 'saturated_fat', 'bad_diet_p'],

        'Physical Activity/Features': ['height', 'weight', 'waist', 'puberty_k', 'sex', 'no_sports_activities_p',
            'birth_weight_p', 'fitbit_resting_hr', 'fitbit_steps', 'fitbit_sedentary_mins', 'fitbit_lightlyactive_mins', 'fitbit_fairlyactive_mins', 'fitbit_veryactive_mins'],



# *****Social Parent/Child (Some Quasi Objective)


        'Technology Use': ['socialmedia_daysperweek_k', 'videogames_daysperweek_k', 'bullied_on_internet_k', 'vgame_thinking'],


        'Religion': [
            'child_religion', 'religious_service_frequency', 'relig_importance'],


        'Family Dynamics & Parenting': ['p_comm_cohesion_ss', 'p_comm_ctrl_ss', 'p_comm_collective_efficacy_ss',
            'fam_fight_often_k', 'fam_no_open_anger_k', 'fam_throw_things_k', 'fam_no_lose_temps_k',
            'fam_criticize_often_k', 'fam_hit_each_other_k', 'fam_keep_peace_k', 'fam_try_one_up_k',
            'fam_no_raise_voices_k', 'family_not_talk_aboutfeelings_p', 'family_peaceful_p',
            'family_open_discussing_anything_p', 'family_lose_temper_rare_p', 'family_believe_not_raise_voice_p',
            'frequent_family_conflict_p', 'family_conflict_ss_p', 'family_expression_ss_p', 'family_intellectual_ss_p',
            'family_activities_ss_p', 'family_organisation_ss_p', 'parents_argue_more_p', 'family_emotionprob_p',
            'parents_divorced_p', 'death_in_family_p', 'family_move_p', 'family_conflict_ss_k',
            'parent_monitoring_ss_k', 'marital_status', 'parent_age', 'sex_P', 'num_brothers_p', 'num_sisters_p',
            'religious_service_frequency', 'relig_importance',  'parent_family_responsibilities_p'],

        'Parent Social Functioning': [
            'parent_bad_relationships_p', 'parent_bad_family_relationship_p', 'parent_not_liked_by_others_p',
            'parent_friendship_trouble_p', 'parent_prefers_older_people_p', 'parent_associates_with_trouble_p',
            'parent_bad_opposite_sex_relationship_p', 'parent_meets_family_duties_p', 'parent_clowns_or_shows_off_p',
            'parent_teases_others_p', 'parent_stands_up_rights_p'
        ],


        'Social Relationship Quality': ['not_liked_p', 'doesnt_get_along_p', 'prosocial_ss_p', 'close_boy_friends_k',
            'close_girl_friends_k', 'peer_net_protective_ss_k', 'peers_beh_prosocial_ss_k', 'peers_beh_delinquent_ss_k',
            'feels_leftout_k', 'not_invited_k', 'excluded_k', 'otherkids_spreadneg_rumors_k', 'otherkids_gossip_k',
            'feels_threatned_k', 'saysmeanthings_others_k', 'otherkids_saymeanthings_k', 'discrimination_ss_k',
            'feels_discriminated_k', 'senses_racism_k', 'doesnt_feel_accepted_k', 'bullied_on_internet_k',
            'prosocial_ss_k', 'socialinfluence_meanfinal_k', 'relational_victimization_ss_k',
            'reputational_aggression_ss_k', 'reputational_victimization_ss_k', 'overt_aggression_ss_k',
            'overt_victimization_ss_k', 'relational_aggression_ss_k', 'peer_net_protective_ss_k',
            'relational_victimization_ss_k', 'overt_aggression_ss_k', 'relational_aggression_ss_k',
            'feels_discriminated_teachers_k', 'feels_discriminated_adults_not_school_k', 'feels_discriminated_students_k',
            'feels_unwanted_american_society_k', 'feels_discriminated_americans_k'],

        'School Dynamics': ['disobeys_at_school_k', 'getalong_teachers_k', 'feelsafe_at_school_k', 'feels_smart_k',
            'enjoys_school_k', 'grades_important_k', 'school_environment_ss_k', 'school_involvement_ss_k',
            'school_disengagement_ss_k', 'bad_grades', 'repeated_grade', 'grades_dropped', 'school_detension_suspension',
            'child_newschool_p', 'finds_schoolboring_k'],

        'Adverse Life Events': ['g_lifeevents_ss_k', 'b_lifeevents_ss_k', 'b_lifeevents_affected_ss_k',
            'experienced_crime_p', 'g_lifeevents_ss_p', 'b_lifeevents_ss_p', 'b_lifeevents_affected_ss_p',
            'car_accident_hurt_p', 'big_accident_need_treatment_p', 'fire_victim_p', 'natural_disaster_victim_p',
            'terrorism_victim_p', 'war_death_witness_p', 'stabbing_shooting_witness_p',
            'stabbing_shooting_victim_community_p', 'stabbing_shooting_victim_home_p', 'beating_victim_home_p',
            'stranger_threatened_child_victim_p', 'family_threatened_child_victim_p', 'adult_family_fighting_victim_p',
            'domestic_child_sexually_abuse_victim_p', 'foreign_child_sexually_abuse_victim_p',
            'peer_child_sexually_abuse_victim_p', 'sudden_death_in_family_p'],


# *****Psycho Child - Bridge in Psycho Parent


        'Cognitive Task Outcomes': ['tb_picvocab', 'tb_picture', 'tb_reading', 'tb_flanker', 'tb_list',
            'tb_cardsort', 'tb_pattern', 'gd_safebets', 'gd_riskybets',
            'ravlt_s_total', 'ravlt_s_repitition', 'ravlt_s_intrusions',
            'ravlt_l_total', 'ravlt_l_repitition', 'ravlt_l_intrusions',
            'nb_correct_nt', 'nb_correct_mrt', 'nb_correct_nt_2back', 'nb_correct_mrt_2back',
            'nb_correct_nt_pos', 'nb_correct_mrt_pos', 'nb_correct_nt_neg', 'nb_correct_mrt_neg',
            'nb2_accuracy_pos', 'nb2_resp_bias_pos', 'nb2_D_prime_pos',
            'nb2_accuracy_neg', 'nb2_resp_bias_neg', 'nb2_D_prime_neg',
            'sst_ssrt_mean_est', 'sst_ssrt_int_est', 'sst_acceptable_performance',
            'mid_mrt_smrw', 'mid_mrt_lgrw', 'mid_total_payout',
            'mid_acceptable_performance', 'mid_num_trials',
            'lmt_accuracy', 'lmt_correct_nt', 'lmt_mrt',
            'lmt_correct_mrt', 'lmt_efficiency'],

        'Sleep Problems': ['difficulty_goingtosleep_p', 'difficulty_wakingup_p', 'nightmares_p', 'fallsleeptime',
            'wakeuptime', 'wakesleepcalc', 'chronotype'],

        'Medical/Somatic Problems': ['medhx_p', 'medhx_doctorvisit_p', 'medhx_emergencyroom_p', 'pain_last_month_k',
            'seriously_sick_lastyear_k', 'body_aches_p', 'frequent_headaches_p', 'nausea_p', 'eye_problems_p',
            'skin_problems_p', 'frequent_stomachaches_p', 'vomiting_p', 'constipated_p', 'bad_toilet_habits_p',
            'wets_bed_p'],

        'Externalizing': ['argues_p', 'stubborn_p', 'temper_tantrums_p', 'bullies_others_p',
            'destroys_own_things_p', 'destroys_others_things_p', 'disobedient_home_p', 'disobedient_school_p',
            'breaks_rules_p', 'fights_p', 'lying_p', 'attacks_others_p', 'steals_home_p', 'steals_outside_p',
            'threatens_others_p', 'whines_p', 'demands_attention_p'],

        'ADHD': ['cant_concentrate_p', 'doesnt_finish_p', 'hyperactive_p', 'impulsive_p', 'easily_distracted_p'],


        'Anxiety': ['social_fear_present_PK', 'worries_p', 'clings_to_adults_p', 'nervous_general_p',
            'nervous_twitching_p', 'fears_excl_school_p', 'fears_school_p', 'fears_being_bad_p', 'paranoid_p'],

        'Other Personality Features': ['easily_offended_p', 'blames_others_p', 'sociable_p', 'school_excitement_p',
            'not_critical_others_p', 'scared_dark_p', 'disagreeable_p', 'goal_continuity_p', 'up_negative_urgency_ss_k',
            'up_lackofplanning_ss_k', 'up_sensationseeking_ss_k', 'up_positiveurgency_ss_k', 'up_lackperseverance_ss_k',
            'bis_behav_inhibition_ss_k', 'bis_reward_responsive_ss_k', 'bis_drive_ss_k', 'bis_funseeking_ss_k',
            'loquacious_p', 'bragadocious_p', 'easily_jealous_p', 'wishes_other_sex_p', 'easily_embarrassed_p',
            'secretive_p', 'perfectionist_p', 'sex_orient_y'],

        'Child Mood Issues': ['enjoys_little_p', 'sad_p', 'suicidal_p', 'guilty_p', 'withdrawn_p'],

        'Child Delta': ["delta_anxdisord_D_p",
            "delta_adhd_D_p", "delta_not_liked_p", "delta_doesnt_get_along_p",
            "delta_family_conflict_ss_p", "delta_family_conflict_ss_k", "delta_bad_grades", "delta_social_problems_D_p",
            "delta_somatic_problems_D_p", "delta_adhd_D_p",
            "delta_bad_diet_p", "delta_atschool_total_problems_ss_t", "delta_b_lifeevents_ss_p"],


# Parent Bridge>>>


        'Parent Delta': [
            'delta_parent_sleep_trouble_p', 'delta_parent_worries_about_family_p',
            'delta_parent_friendship_trouble_p', 'delta_parent_poor_work_performance_p',
            'delta_parent_aches_pains_p', 'delta_parent_not_liked_by_others_p',
            'delta_parent_feels_overwhelmed_p', 'delta_parent_feels_unloved_p',
            'delta_parent_bad_family_relationship_p', 'delta_parent_worries_about_future_p',
            'delta_parent_worries_a_lot_p', 'delta_parent_depressed_p',
            'delta_parent_concentration_trouble_p', 'delta_parent_stubborn_irritable_p',
            'delta_parent_drinks_too_much_p', 'delta_parent_financial_failures_p',
            'delta_parent_meets_family_duties_p', 'delta_parent_planning_trouble_p',
            'delta_parent_bad_relationships_p', 'delta_parent_drug_use_p'
        ],

        'Parent Mood Issues': [
            'parent_cries_a_lot_p', 'parent_lonely_p', 'parent_feels_unloved_p', 'parent_paranoid_p',
            'parent_feels_inferior_p', 'parent_depressed_p', 'parent_feels_unsuccessful_p',
            'parent_tired_no_reason_p', 'parent_low_energy_p', 'parent_sleep_trouble_p',
            'parent_enjoys_little_p', 'parent_sudden_mood_changes_p', 'parent_suicidal_thoughts_p', 'parent_happy_person_p'
        ],

        'Parent Anxiety': [
            'parent_fearful_or_anxious_p', 'parent_specific_fears_p', 'parent_fear_of_bad_thoughts_p',
            'parent_worries_about_future_p', 'parent_worries_about_family_p', 'parent_worries_a_lot_p',
            'parent_relationship_concerns_p'
        ],

        'Parent Cognitive and Attention Issues': [
            'parent_forgetful_p', 'parent_concentration_trouble_p', 'parent_confused_p', 'parent_planning_trouble_p',
            'parent_not_good_at_details_p', 'parent_obsessive_thoughts_p',
            'parent_repeats_acts_p', 'parent_max_effort_p', 'parent_disorganized_p', 'parent_loses_things_p',
            'parent_decision_trouble_p', 'parent_priority_trouble_p'
        ],

        'Parent Personality': [
            'parent_bragging_p', 'parent_honest_p', 'parent_secretive_p', 'parent_stubborn_irritable_p',
            'parent_clumsy_p', 'parent_strange_thoughts_p', 'parent_self_conscious_p', 'parent_uses_opportunities_p',
            'parent_louder_than_others_p', 'parent_yells_a_lot_p', 'parent_shy_or_timid_p', 'parent_restless_p',
            'parent_easily_bored_p', 'parent_hyperactive_p', 'parent_talks_too_much_p', 'parent_avoids_talking_p',
            'parent_prefers_to_be_alone_p', 'parent_no_guilt_p', 'parent_sense_of_fairness_p',
            'parent_high_sleep_duration_p'
        ],


        'Family Drug Use': [
          'hallucinogen_use_history_B_p', 'hallucinogen_current_B_p', 'sedative_hypnotic_anxiolytic_use_B_p',
          'father_alcohol', 'mother_alcohol', 'father_druguse', 'mother_druguse',
          'cigs_during_pregnancy_p', 'alcohol_during_pregnancy_p', 'weed_during_pregnancy_p',
          'cocaine_during_pregnancy_p', 'heroin_during_pregnancy_p',
          'prescriptionmed_pregnancy_p', 'cigs_before_pregnancy_p', 'alcohol_before_pregnancy_p',
          'weed_before_pregnancy_p', 'cocaine_before_pregnancy_p', 'heroin_before_pregnancy_p',
          'drugs_before_pregnancy_p', 'drinksperweek_during_pregnancy_p', 'drugs_during_pregnancy_p',
          'caffeine_during_pregnancy_p', 'parent_tobacco_use_frequency_p', 'parent_drug_use_p', 'parent_drinks_too_much_p',
          'parent_drinks_frequency_p', 'parent_drunk_days_p', 'parent_drug_days_nonmedical_p'],

       'SES & Mobility': ['parent_education', 'parent_income', 'struggle_food_expenses', 'positive_finance_p', 'parent_work_absences_p', 'parent_financial_trouble_p', 'parent_fails_to_pay_debts_p'],

       'Dynamic Cognitive Control Parameters': ['sst_mean_absdelta', 'sst_dG', 'sst_theta1', 'sst_tG', 'sst_mu', 'sst_aG1', 'sst_median_absdelta', 'sst_kappa0', 'sst_gamma0', 'sst_pp', 'sst_aG2', 'sst_aS', 'sst_dS', 'sst_absdeltaCV', 'sst_pdrgCV', 'sst_mean_CV', 'sst_mean_PDR', 'sst_sM', 'sst_absdeltaMax', 'sst_median_ssrt', 'sst_bG', 'sst_betasCV', 'sst_absdeltaRV'],

# ***** Bridge back into Bio Ojective.

}

# mapping for variable type

type_map = {
  'Objective Bio genes_neuroimaging': [
    # Structural
    'pc_struct1', 'pc_struct2', 'pc_struct3', 'pc_struct4', 'pc_struct5', 'pc_struct6', 'pc_struct7', 'pc_struct8',
    'pc_struct9', 'pc_struct10', 'pc_struct11', 'pc_struct12', 'pc_struct13', 'pc_struct14', 'pc_struct15',
    'pc_struct16', 'pc_struct17', 'pc_struct18', 'pc_struct19', 'pc_struct20', 'pc_struct21', 'pc_struct22',
    'pc_struct23', 'pc_struct24', 'pc_struct25', 'pc_struct26', 'pc_struct27', 'pc_struct28', 'pc_struct29',
    'pc_struct30', 'pc_struct31', 'pc_struct32', 'pc_struct33', 'pc_struct34', 'pc_struct35', 'pc_struct36',
    'pc_struct37', 'pc_struct38', 'pc_struct39', 'pc_struct40', 'pc_struct41', 'pc_struct42', 'pc_struct43',
    'pc_struct44', 'pc_struct45', 'pc_struct46', 'pc_struct47', 'pc_struct48', 'pc_struct49', 'pc_struct50',
    'pc_struct51', 'pc_struct52', 'pc_struct53', 'pc_struct54', 'pc_struct55', 'pc_struct56', 'pc_struct57',
    'pc_struct58', 'pc_struct59', 'pc_struct60', 'pc_struct61', 'pc_struct62', 'pc_struct63', 'pc_struct64',
    'pc_struct65', 'pc_struct66', 'pc_struct67', 'pc_struct68', 'pc_struct69', 'pc_struct70', 'pc_struct71',
    'pc_struct72', 'pc_struct73', 'pc_struct74', 'pc_struct75',

    # DTI
    'pc_DTI1', 'pc_DTI2', 'pc_DTI3', 'pc_DTI4', 'pc_DTI5', 'pc_DTI6', 'pc_DTI7', 'pc_DTI8', 'pc_DTI9', 'pc_DTI10',
    'pc_DTI11', 'pc_DTI12', 'pc_DTI13', 'pc_DTI14', 'pc_DTI15', 'pc_DTI16', 'pc_DTI17', 'pc_DTI18', 'pc_DTI19',
    'pc_DTI20', 'pc_DTI21', 'pc_DTI22', 'pc_DTI23', 'pc_DTI24', 'pc_DTI25', 'pc_DTI26', 'pc_DTI27', 'pc_DTI28',
    'pc_DTI29', 'pc_DTI30', 'pc_DTI31', 'pc_DTI32', 'pc_DTI33', 'pc_DTI34', 'pc_DTI35', 'pc_DTI36', 'pc_DTI37',
    'pc_DTI38', 'pc_DTI39', 'pc_DTI40', 'pc_DTI41', 'pc_DTI42', 'pc_DTI43', 'pc_DTI44', 'pc_DTI45', 'pc_DTI46',
    'pc_DTI47', 'pc_DTI48', 'pc_DTI49', 'pc_DTI50', 'pc_DTI51', 'pc_DTI52', 'pc_DTI53', 'pc_DTI54', 'pc_DTI55',
    'pc_DTI56', 'pc_DTI57', 'pc_DTI58', 'pc_DTI59', 'pc_DTI60', 'pc_DTI61', 'pc_DTI62', 'pc_DTI63', 'pc_DTI64',
    'pc_DTI65', 'pc_DTI66', 'pc_DTI67', 'pc_DTI68', 'pc_DTI69', 'pc_DTI70', 'pc_DTI71', 'pc_DTI72', 'pc_DTI73',
    'pc_DTI74', 'pc_DTI75',

    # Resting State
    'pc_rsFMRI1', 'pc_rsFMRI2', 'pc_rsFMRI3', 'pc_rsFMRI4', 'pc_rsFMRI5', 'pc_rsFMRI6', 'pc_rsFMRI7', 'pc_rsFMRI8',
    'pc_rsFMRI9', 'pc_rsFMRI10', 'pc_rsFMRI11', 'pc_rsFMRI12', 'pc_rsFMRI13', 'pc_rsFMRI14', 'pc_rsFMRI15',
    'pc_rsFMRI16', 'pc_rsFMRI17', 'pc_rsFMRI18', 'pc_rsFMRI19', 'pc_rsFMRI20', 'pc_rsFMRI21', 'pc_rsFMRI22',
    'pc_rsFMRI23', 'pc_rsFMRI24', 'pc_rsFMRI25', 'pc_rsFMRI26', 'pc_rsFMRI27', 'pc_rsFMRI28', 'pc_rsFMRI29',
    'pc_rsFMRI30', 'pc_rsFMRI31', 'pc_rsFMRI32', 'pc_rsFMRI33', 'pc_rsFMRI34', 'pc_rsFMRI35', 'pc_rsFMRI36',
    'pc_rsFMRI37', 'pc_rsFMRI38', 'pc_rsFMRI39', 'pc_rsFMRI40', 'pc_rsFMRI41', 'pc_rsFMRI42', 'pc_rsFMRI43',
    'pc_rsFMRI44', 'pc_rsFMRI45', 'pc_rsFMRI46', 'pc_rsFMRI47', 'pc_rsFMRI48', 'pc_rsFMRI49', 'pc_rsFMRI50',
    'pc_rsFMRI51', 'pc_rsFMRI52', 'pc_rsFMRI53', 'pc_rsFMRI54', 'pc_rsFMRI55', 'pc_rsFMRI56', 'pc_rsFMRI57',
    'pc_rsFMRI58', 'pc_rsFMRI59', 'pc_rsFMRI60', 'pc_rsFMRI61', 'pc_rsFMRI62', 'pc_rsFMRI63', 'pc_rsFMRI64',
    'pc_rsFMRI65', 'pc_rsFMRI66', 'pc_rsFMRI67', 'pc_rsFMRI68', 'pc_rsFMRI69', 'pc_rsFMRI70', 'pc_rsFMRI71',
    'pc_rsFMRI72', 'pc_rsFMRI73', 'pc_rsFMRI74', 'pc_rsFMRI75',

    # Task fMRI
    'pc_SSTmri1', 'pc_SSTmri2', 'pc_SSTmri3', 'pc_SSTmri4', 'pc_SSTmri5', 'pc_SSTmri6', 'pc_SSTmri7', 'pc_SSTmri8',
    'pc_SSTmri9', 'pc_SSTmri10', 'pc_SSTmri11', 'pc_SSTmri12', 'pc_SSTmri13', 'pc_SSTmri14', 'pc_SSTmri15', 'pc_SSTmri16',
    'pc_SSTmri17', 'pc_SSTmri18', 'pc_SSTmri19', 'pc_SSTmri20', 'pc_SSTmri21', 'pc_SSTmri22', 'pc_SSTmri23', 'pc_SSTmri24',
    'pc_SSTmri25', 'pc_SSTmri26', 'pc_SSTmri27', 'pc_SSTmri28', 'pc_SSTmri29', 'pc_SSTmri30', 'pc_SSTmri31', 'pc_SSTmri32',
    'pc_SSTmri33', 'pc_SSTmri34', 'pc_SSTmri35', 'pc_SSTmri36', 'pc_SSTmri37', 'pc_SSTmri38', 'pc_SSTmri39', 'pc_SSTmri40',
    'pc_SSTmri41', 'pc_SSTmri42', 'pc_SSTmri43', 'pc_SSTmri44', 'pc_SSTmri45', 'pc_SSTmri46', 'pc_SSTmri47', 'pc_SSTmri48',
    'pc_SSTmri49', 'pc_SSTmri50', 'pc_SSTmri51', 'pc_SSTmri52', 'pc_SSTmri53', 'pc_SSTmri54', 'pc_SSTmri55', 'pc_SSTmri56',
    'pc_SSTmri57', 'pc_SSTmri58', 'pc_SSTmri59', 'pc_SSTmri60', 'pc_SSTmri61', 'pc_SSTmri62', 'pc_SSTmri63', 'pc_SSTmri64',
    'pc_SSTmri65', 'pc_SSTmri66', 'pc_SSTmri67', 'pc_SSTmri68', 'pc_SSTmri69', 'pc_SSTmri70', 'pc_SSTmri71', 'pc_SSTmri72',
    'pc_SSTmri73', 'pc_SSTmri74', 'pc_SSTmri75', 'pc_nbackmri1', 'pc_nbackmri2', 'pc_nbackmri3', 'pc_nbackmri4', 'pc_nbackmri5',
    'pc_nbackmri6', 'pc_nbackmri7', 'pc_nbackmri8', 'pc_nbackmri9', 'pc_nbackmri10', 'pc_nbackmri11', 'pc_nbackmri12',
    'pc_nbackmri13', 'pc_nbackmri14', 'pc_nbackmri15', 'pc_nbackmri16', 'pc_nbackmri17', 'pc_nbackmri18', 'pc_nbackmri19',
    'pc_nbackmri20', 'pc_nbackmri21', 'pc_nbackmri22', 'pc_nbackmri23', 'pc_nbackmri24', 'pc_nbackmri25', 'pc_nbackmri26',
    'pc_nbackmri27', 'pc_nbackmri28', 'pc_nbackmri29', 'pc_nbackmri30', 'pc_nbackmri31', 'pc_nbackmri32', 'pc_nbackmri33',
    'pc_nbackmri34', 'pc_nbackmri35', 'pc_nbackmri36', 'pc_nbackmri37', 'pc_nbackmri38', 'pc_nbackmri39', 'pc_nbackmri40',
    'pc_nbackmri41', 'pc_nbackmri42', 'pc_nbackmri43', 'pc_nbackmri44', 'pc_nbackmri45', 'pc_nbackmri46', 'pc_nbackmri47',
    'pc_nbackmri48', 'pc_nbackmri49', 'pc_nbackmri50', 'pc_nbackmri51', 'pc_nbackmri52', 'pc_nbackmri53', 'pc_nbackmri54',
    'pc_nbackmri55', 'pc_nbackmri56', 'pc_nbackmri57', 'pc_nbackmri58', 'pc_nbackmri59', 'pc_nbackmri60', 'pc_nbackmri61',
    'pc_nbackmri62', 'pc_nbackmri63', 'pc_nbackmri64', 'pc_nbackmri65', 'pc_nbackmri66', 'pc_nbackmri67', 'pc_nbackmri68',
    'pc_nbackmri69', 'pc_nbackmri70', 'pc_nbackmri71', 'pc_nbackmri72', 'pc_nbackmri73', 'pc_nbackmri74', 'pc_nbackmri75',
    'pc_midfmri1', 'pc_midfmri2', 'pc_midfmri3', 'pc_midfmri4', 'pc_midfmri5', 'pc_midfmri6', 'pc_midfmri7', 'pc_midfmri8',
    'pc_midfmri9', 'pc_midfmri10', 'pc_midfmri11', 'pc_midfmri12', 'pc_midfmri13', 'pc_midfmri14', 'pc_midfmri15',
    'pc_midfmri16', 'pc_midfmri17', 'pc_midfmri18', 'pc_midfmri19', 'pc_midfmri20', 'pc_midfmri21', 'pc_midfmri22',
    'pc_midfmri23', 'pc_midfmri24', 'pc_midfmri25', 'pc_midfmri26', 'pc_midfmri27', 'pc_midfmri28', 'pc_midfmri29',
    'pc_midfmri30', 'pc_midfmri31', 'pc_midfmri32', 'pc_midfmri33', 'pc_midfmri34', 'pc_midfmri35', 'pc_midfmri36',
    'pc_midfmri37', 'pc_midfmri38', 'pc_midfmri39', 'pc_midfmri40', 'pc_midfmri41', 'pc_midfmri42', 'pc_midfmri43',
    'pc_midfmri44', 'pc_midfmri45', 'pc_midfmri46', 'pc_midfmri47', 'pc_midfmri48', 'pc_midfmri49', 'pc_midfmri50',
    'pc_midfmri51', 'pc_midfmri52', 'pc_midfmri53', 'pc_midfmri54', 'pc_midfmri55', 'pc_midfmri56', 'pc_midfmri57',
    'pc_midfmri58', 'pc_midfmri59', 'pc_midfmri60', 'pc_midfmri61', 'pc_midfmri62', 'pc_midfmri63', 'pc_midfmri64',
    'pc_midfmri65', 'pc_midfmri66', 'pc_midfmri67', 'pc_midfmri68', 'pc_midfmri69', 'pc_midfmri70', 'pc_midfmri71',
    'pc_midfmri72', 'pc_midfmri73', 'pc_midfmri74', 'pc_midfmri75',

    # Ethnicity/Nationality
    'pc_gene_aces1', 'pc_gene_aces2', 'pc_gene_aces3', 'pc_gene_aces4', 'pc_gene_aces5', 'pc_gene_aces6',
    'pc_gene_aces7', 'pc_gene_aces8', 'pc_gene_aces9', 'pc_gene_aces10', 'pc_gene_aces11', 'pc_gene_aces12',
    'pc_gene_aces13', 'pc_gene_aces14', 'pc_gene_aces15', 'pc_gene_aces16', 'pc_gene_aces17', 'pc_gene_aces18',
    'pc_gene_aces19', 'pc_gene_aces20', 'pc_gene_aces21', 'pc_gene_aces22', 'pc_gene_aces23', 'pc_gene_aces24',
    'pc_gene_aces25', 'pc_gene_aces26', 'pc_gene_aces27', 'pc_gene_aces28', 'pc_gene_aces29', 'pc_gene_aces30',
    'pc_gene_aces31', 'pc_gene_aces32', 'desc_african_AFR_B', 'desc_native_american_AMR_B',
    'desc_alaska_native_AMR_B', 'desc_chinese_EAS_B', 'desc_japanese_EAS_B', 'desc_korean_EAS_B',
    'desc_vietnamese_EAS_B', 'desc_european_EUR_B', 'desc_asian_indian_SAS_B', 'desc_other_south_asian_SAS_B',
    'desc_latin_B',

    # Physical Measurements
    'height', 'weight', 'waist', 'sex', 'no_sports_activities_p', 'birth_weight_p', 'fitbit_resting_hr',
    'fitbit_steps', 'fitbit_sedentary_mins', 'fitbit_lightlyactive_mins', 'fitbit_fairlyactive_mins',
    'fitbit_veryactive_mins'],


    'Objective Non Bio resid_diet_ses_cog': [
    # Residential Characteristics
    'neighborhood_safety_ss_p', 'neighborhood_safe_y', 'resid_density', 'resid_walkability', 'resid_prox_roads',
    'resid_crime_tot', 'resid_crime_violent', 'resid_crime_drug', 'resid_crime_dui', 'resid_lead_risk_poverty',
    'resid_lead_risk_houses_perc', 'resid_lead_risk', 'resid_no2_avg', 'resid_pm25_avg', 'resid_sexism',
    'resid_sex_orient_bias', 'resid_immigrant_bias', 'resid_racism', 'L_site_id',

    # Diet
    'fruit_intake', 'vegetable_intake', 'protein_sources_intake', 'legume_intake', 'added_sugar',
    'sugary_beverage_freq', 'dairy_intake', 'whole_grain_intake', 'total_calories', 'protein_intake',
    'carbohydrate_intake', 'fiber_intake', 'sodium_intake', 'potassium_intake', 'total_sugar', 'saturated_fat',

    # SES
    'parent_education', 'parent_income', 'struggle_food_expenses', 'positive_finance_p', 'parent_work_absences_p',
    'parent_financial_trouble_p', 'parent_fails_to_pay_debts_p', 'marital_status', 'parent_age', 'sex_P',
    'num_brothers_p', 'num_sisters_p',

    # Cognitive Task Outcomes
    'tb_picvocab', 'tb_picture', 'tb_reading', 'tb_flanker', 'tb_list', 'tb_cardsort', 'tb_pattern',
    'gd_safebets', 'gd_riskybets', 'ravlt_s_total', 'ravlt_s_repitition', 'ravlt_s_intrusions',
    'ravlt_l_total', 'ravlt_l_repitition', 'ravlt_l_intrusions', 'nb_correct_nt', 'nb_correct_mrt',
    'nb_correct_nt_2back', 'nb_correct_mrt_2back', 'nb_correct_nt_pos', 'nb_correct_mrt_pos', 'nb_correct_nt_neg',
    'nb_correct_mrt_neg', 'nb2_accuracy_pos', 'nb2_resp_bias_pos', 'nb2_D_prime_pos', 'nb2_accuracy_neg',
    'nb2_resp_bias_neg', 'nb2_D_prime_neg', 'sst_ssrt_mean_est', 'sst_ssrt_int_est', 'sst_acceptable_performance',
    'mid_mrt_smrw', 'mid_mrt_lgrw', 'mid_total_payout', 'mid_acceptable_performance', 'mid_num_trials',
    'lmt_accuracy', 'lmt_correct_nt', 'lmt_mrt', 'lmt_correct_mrt', 'lmt_efficiency',

    # Religion
    'child_religion', 'religious_service_frequency', 'relig_importance',

    # Education
    'bad_grades'],

    "Parent-Child Mood Ratings": ['enjoys_little_p', 'sad_p', 'suicidal_p', 'guilty_p', 'withdrawn_p'],

    'Child Self-Report School, Family, and Social': [
    # School Dynamics
    'disobeys_at_school_k', 'getalong_teachers_k', 'feelsafe_at_school_k', 'feels_smart_k', 'enjoys_school_k',
    'grades_important_k', 'school_environment_ss_k', 'school_involvement_ss_k', 'school_disengagement_ss_k',
    'repeated_grade', 'child_newschool_p', 'finds_schoolboring_k',

    # Community Dynamics
    'p_comm_cohesion_ss', 'p_comm_ctrl_ss', 'p_comm_collective_efficacy_ss',

    # Family Conflict
    'fam_fight_often_k', 'fam_no_open_anger_k', 'fam_throw_things_k', 'fam_no_lose_temps_k', 'fam_criticize_often_k',
    'fam_hit_each_other_k', 'fam_keep_peace_k', 'fam_try_one_up_k', 'fam_no_raise_voices_k'

    #social quality
    'prosocial_ss_p', 'close_boy_friends_k',
            'close_girl_friends_k', 'peer_net_protective_ss_k', 'peers_beh_prosocial_ss_k', 'peers_beh_delinquent_ss_k',
            'feels_leftout_k', 'not_invited_k', 'excluded_k', 'otherkids_spreadneg_rumors_k', 'otherkids_gossip_k',
            'feels_threatned_k', 'saysmeanthings_others_k', 'otherkids_saymeanthings_k', 'discrimination_ss_k',
            'feels_discriminated_k', 'senses_racism_k', 'doesnt_feel_accepted_k', 'bullied_on_internet_k',
            'prosocial_ss_k', 'socialinfluence_meanfinal_k', 'relational_victimization_ss_k',
            'reputational_aggression_ss_k', 'reputational_victimization_ss_k', 'overt_aggression_ss_k',
            'overt_victimization_ss_k', 'relational_aggression_ss_k', 'peer_net_protective_ss_k',
            'relational_victimization_ss_k', 'overt_aggression_ss_k', 'relational_aggression_ss_k',
            'feels_discriminated_teachers_k', 'feels_discriminated_adults_not_school_k', 'feels_discriminated_students_k',
            'feels_unwanted_american_society_k', 'feels_discriminated_americans_k'
    #Personality
 'up_negative_urgency_ss_k',
            'up_lackofplanning_ss_k', 'up_sensationseeking_ss_k', 'up_positiveurgency_ss_k', 'up_lackperseverance_ss_k',
            'bis_behav_inhibition_ss_k', 'bis_reward_responsive_ss_k', 'bis_drive_ss_k', 'bis_funseeking_ss_k'],

  'Parent on Child Psychopathology Ratings': [
    # General
    'not_liked_p', 'doesnt_get_along_p',

    # Anxiety
    'social_fear_present_PK', 'worries_p', 'clings_to_adults_p', 'nervous_general_p', 'nervous_twitching_p',
    'fears_excl_school_p', 'fears_school_p', 'fears_being_bad_p', 'paranoid_p', 'fear_becoming_obese_present_PK',
    'self_conscious_k', 'anxious_fearful_k',

    # ADHD
    'trouble_concentrating_k', 'difficulty_finishing_tasks_k', 'easily_distracted_k', 'cant_concentrate_p',
    'doesnt_finish_p', 'hyperactive_p', 'impulsive_p', 'easily_distracted_p',

    # Externalizing
    'argues_p', 'stubborn_p', 'temper_tantrums_p', 'bullies_others_p', 'destroys_own_things_p', 'destroys_others_things_p',
    'disobedient_home_p', 'disobedient_school_p', 'breaks_rules_p', 'fights_p', 'lying_p', 'attacks_others_p',
    'steals_home_p', 'steals_outside_p', 'threatens_others_p', 'whines_p', 'demands_attention_p',
    'conduct_physical_fights_present_PK', 'acts_immature_k', 'destroys_others_things_k', 'disobeys_parents_k',
    'stubborn_k', 'hot_temper_k',

    # Other Personality Features
    'easily_offended_p', 'blames_others_p', 'sociable_p', 'school_excitement_p', 'not_critical_others_p',
    'scared_dark_p', 'disagreeable_p', 'goal_continuity_p', 'loquacious_p', 'bragadocious_p', 'easily_jealous_p',
    'wishes_other_sex_p', 'easily_embarrassed_p', 'secretive_p', 'perfectionist_p',

    # Medical/Somatic Problems
    'medhx_p', 'medhx_doctorvisit_p', 'medhx_emergencyroom_p', 'pain_last_month_k', 'seriously_sick_lastyear_k',
    'body_aches_p', 'frequent_headaches_p', 'nausea_p', 'eye_problems_p', 'skin_problems_p', 'frequent_stomachaches_p',
    'vomiting_p', 'constipated_p', 'bad_toilet_habits_p', 'wets_bed_p'],


     'Adverse Life Events': ['experienced_crime_p', 'g_lifeevents_ss_p', 'b_lifeevents_ss_p', 'b_lifeevents_affected_ss_p',
            'car_accident_hurt_p', 'big_accident_need_treatment_p', 'fire_victim_p', 'natural_disaster_victim_p',
            'terrorism_victim_p', 'war_death_witness_p', 'stabbing_shooting_witness_p',
            'stabbing_shooting_victim_community_p', 'stabbing_shooting_victim_home_p', 'beating_victim_home_p',
            'stranger_threatened_child_victim_p', 'family_threatened_child_victim_p', 'adult_family_fighting_victim_p',
            'domestic_child_sexually_abuse_victim_p', 'foreign_child_sexually_abuse_victim_p',
            'peer_child_sexually_abuse_victim_p', 'sudden_death_in_family_p',
            'g_lifeevents_ss_k', 'b_lifeevents_ss_k', 'b_lifeevents_affected_ss_k'],


 'Parent Psychopathology Self-Report': [
    # Parent Anxiety
    'parent_fearful_or_anxious_p', 'parent_specific_fears_p', 'parent_fear_of_bad_thoughts_p',
    'parent_worries_about_future_p', 'parent_worries_about_family_p', 'parent_worries_a_lot_p',
    'parent_relationship_concerns_p',

    # Parent Mood Issues
    'parent_cries_a_lot_p', 'parent_lonely_p', 'parent_feels_unloved_p', 'parent_paranoid_p',
    'parent_feels_inferior_p', 'parent_depressed_p', 'parent_feels_unsuccessful_p', 'parent_tired_no_reason_p',
    'parent_low_energy_p', 'parent_sleep_trouble_p', 'parent_enjoys_little_p', 'parent_sudden_mood_changes_p',
    'parent_suicidal_thoughts_p', 'parent_happy_person_p',

    # Parent Impulsivity and Behavior Regulation
    'parent_impulsive_p', 'parent_risky_decisions_p', 'parent_drives_too_fast_p', 'parent_tardy_often_p',
    'parent_money_management_trouble_p', 'parent_priority_trouble_p', 'parent_behavior_changeable_p',
    'parent_hot_temper_p', 'parent_attention_seeking_p', 'parent_destroys_own_things_p',
    'parent_destroys_others_things_p', 'parent_doesnt_finish_tasks_p', 'parent_strange_behavior_p',
    'parent_illegal_behavior_p', 'parent_self_harm_p', 'parent_doesnt_eat_well_p',

    # Parent Cognitive and Attention Issues
    'parent_forgetful_p', 'parent_concentration_trouble_p', 'parent_confused_p', 'parent_planning_trouble_p',
    'parent_not_good_at_details_p', 'parent_obsessive_thoughts_p', 'parent_repeats_acts_p',
    'parent_max_effort_p', 'parent_disorganized_p', 'parent_loses_things_p', 'parent_decision_trouble_p',
    'parent_priority_trouble_p',

    # Parent Personality
    'parent_bragging_p', 'parent_honest_p', 'parent_secretive_p', 'parent_stubborn_irritable_p', 'parent_clumsy_p',
    'parent_strange_thoughts_p', 'parent_self_conscious_p', 'parent_uses_opportunities_p', 'parent_louder_than_others_p',
    'parent_yells_a_lot_p', 'parent_shy_or_timid_p', 'parent_restless_p', 'parent_easily_bored_p',
    'parent_hyperactive_p', 'parent_talks_too_much_p', 'parent_avoids_talking_p', 'parent_prefers_to_be_alone_p',
    'parent_no_guilt_p', 'parent_sense_of_fairness_p', 'parent_high_sleep_duration_p',

    # Parent Other
    'parent_physical_attacks_p', 'parent_picks_skin_p', 'parent_heart_racing_p', 'parent_numbness_p',
    'parent_sees_things_p', 'parent_hears_voices_p', 'parent_speech_problems_p', 'parent_opposite_sex_wish_p',

    # Parent Social Functioning
    'parent_bad_relationships_p', 'parent_bad_family_relationship_p', 'parent_not_liked_by_others_p',
    'parent_friendship_trouble_p', 'parent_prefers_older_people_p', 'parent_associates_with_trouble_p',
    'parent_bad_opposite_sex_relationship_p', 'parent_meets_family_duties_p', 'parent_clowns_or_shows_off_p',
    'parent_teases_others_p', 'parent_stands_up_rights_p'],



    'Parent Self-Report Damily Dynamics and Drug Use': [
    # Family Dynamics
    'family_not_talk_aboutfeelings_p', 'family_peaceful_p', 'family_open_discussing_anything_p',
    'family_lose_temper_rare_p', 'family_believe_not_raise_voice_p', 'frequent_family_conflict_p',
    'family_conflict_ss_p', 'family_expression_ss_p', 'family_intellectual_ss_p', 'family_activities_ss_p',
    'family_organisation_ss_p', 'parents_divorced_p', 'death_in_family_p', 'family_move_p', 'family_conflict_ss_k',
    'parent_monitoring_ss_k', 'parent_family_responsibilities_p',

    # Drug Use and Substance History
    'hallucinogen_use_history_B_p', 'hallucinogen_current_B_p', 'sedative_hypnotic_anxiolytic_use_B_p',
    'father_alcohol', 'mother_alcohol', 'father_druguse', 'mother_druguse', 'cigs_during_pregnancy_p',
    'alcohol_during_pregnancy_p', 'weed_during_pregnancy_p', 'cocaine_during_pregnancy_p', 'heroin_during_pregnancy_p',
    'prescriptionmed_pregnancy_p', 'cigs_before_pregnancy_p', 'alcohol_before_pregnancy_p', 'weed_before_pregnancy_p',
    'cocaine_before_pregnancy_p', 'heroin_before_pregnancy_p', 'drugs_before_pregnancy_p',
    'drinksperweek_during_pregnancy_p', 'drugs_during_pregnancy_p', 'caffeine_during_pregnancy_p',
    'parent_tobacco_use_frequency_p', 'parent_drug_use_p', 'parent_drinks_too_much_p', 'parent_drinks_frequency_p',
    'parent_drunk_days_p', 'parent_drug_days_nonmedical_p'],

    'Dynamic Cognitive Control Parameters': ['sst_mean_absdelta', 'sst_dG', 'sst_theta1', 'sst_tG', 'sst_mu', 'sst_aG1', 'sst_median_absdelta', 'sst_kappa0', 'sst_gamma0', 'sst_pp', 'sst_aG2', 'sst_aS', 'sst_dS', 'sst_absdeltaCV', 'sst_pdrgCV', 'sst_mean_CV', 'sst_mean_PDR', 'sst_sM', 'sst_absdeltaMax', 'sst_median_ssrt', 'sst_bG', 'sst_betasCV', 'sst_absdeltaRV'],
}

type_map = {k: set(v) for k, v in type_map.items()}

def match_kind(var):
  for k in type_map:
    if var in type_map[k]:
      return k
  return None

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# This file creates hierarchy, vertex, and connection dataframes to be imported
# into an R script using ggraph and igraph, in the format that graph_from_data_frame() expects

In [ ]:
data = pd.read_csv("/content/CLEAN_ABCD_5.1_panel_20250321.csv") # load extracted data
data = data[data["time"] == 2]

<ipython-input-6-01974d1de614>:1: DtypeWarning: Columns (316,320) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/CLEAN_ABCD_5.1_panel_20250321.csv")


In [ ]:
flattened = list(set(sum(variable_groups.values(), []))) # get all variables
with_dupes = sum(variable_groups.values(), [])

In [ ]:
numeric = data.select_dtypes(include='number')[[f for f in flattened if f in data.columns]] # get numeric variables

In [ ]:
# check for duplicates

duplicate_set = set()
d_seen = set()

for x in with_dupes:
    if x in d_seen:
        duplicate_set.add(x)
    else:
        d_seen.add(x)

duplicate_set

In [ ]:
corr_matrix = numeric.corr() # calculate correlation matrix

In [ ]:
df = corr_matrix
df.insert(0, "DummyColumn", 0)

In [ ]:
df

,DummyColumn,disobeys_at_school_k,doesnt_feel_accepted_k,peer_net_protective_ss_k,family_conflict_ss_p,nb2_accuracy_pos,up_lackperseverance_ss_k,pc_gene_aces15,fitbit_sedentary_mins,sst_acceptable_performance,...,getalong_teachers_k,parent_tired_no_reason_p,delta_parent_bad_family_relationship_p,parent_enjoys_little_p,b_lifeevents_affected_ss_k,child_newschool_p,parent_priority_trouble_p,fitbit_fairlyactive_mins,sst_dS,not_invited_k
disobeys_at_school_k,0,1.000000,0.059212,0.019142,0.069792,-0.024425,0.191779,-0.027372,0.026246,NaN,...,-0.242741,0.030353,-0.015554,0.042735,0.149139,0.011251,0.037396,0.039288,-0.046372,0.071437
doesnt_feel_accepted_k,0,0.059212,1.000000,0.045531,0.010616,-0.034721,0.052266,-0.001141,0.061508,NaN,...,-0.062386,0.013789,0.007606,0.018758,0.109805,-0.010365,-0.003201,-0.023679,-0.029901,0.109600
peer_net_protective_ss_k,0,0.019142,0.045531,1.000000,-0.005927,-0.000243,-0.084707,-0.010609,0.032191,NaN,...,0.032232,-0.012181,0.029051,0.003953,0.182572,-0.036372,-0.016121,0.033243,0.025641,0.085006
family_conflict_ss_p,0,0.069792,0.010616,-0.005927,1.000000,0.007523,0.070243,-0.006895,-0.019806,NaN,...,-0.048222,0.167349,0.057893,0.174199,0.064593,0.039764,0.208914,0.038965,-0.034319,0.051379
nb2_accuracy_pos,0,-0.024425,-0.034721,-0.000243,0.007523,1.000000,-0.076483,0.006607,-0.039346,NaN,...,0.022377,-0.027257,0.012208,-0.047037,-0.051274,0.012627,-0.016564,0.008523,0.168473,-0.016724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
child_newschool_p,0,0.011251,-0.010365,-0.036372,0.039764,0.012627,0.025957,-0.005586,0.003975,NaN,...,-0.018998,0.046635,0.012781,0.023767,0.027728,1.000000,0.052839,-0.048799,0.004318,0.017375
parent_priority_trouble_p,0,0.037396,-0.003201,-0.016121,0.208914,-0.016564,0.076143,0.012632,-0.001681,NaN,...,-0.031982,0.298054,0.063228,0.261874,0.062278,0.052839,1.000000,-0.006410,-0.018024,0.036920
fitbit_fairlyactive_mins,0,0.039288,-0.023679,0.033243,0.038965,0.008523,-0.050980,-0.000757,-0.244702,NaN,...,-0.012268,-0.030437,0.021035,-0.022369,0.018492,-0.048799,-0.006410,1.000000,0.022481,-0.015936
sst_dS,0,-0.046372,-0.029901,0.025641,-0.034319,0.168473,-0.096221,-0.004128,-0.062503,NaN,...,0.035878,-0.016087,0.000712,-0.014218,-0.060902,0.004318,-0.018024,0.022481,1.000000,-0.024467


In [ ]:
# remove duplicates

no_dupes_vars = {}
seen_vars = set()

for v, k in variable_groups.items():
  new_vars = set(k) - seen_vars
  if new_vars:
    no_dupes_vars[v] = list(new_vars)
    seen_vars.update(new_vars)

In [ ]:
ggplot_colors = [
    "#F8766D", "#B79F00", "#00BA38", "#00BFC4", "#619CFF", "#F564E3",
    "#00A9FF", "#FF61CC", "#E76BF3", "#00C08B", "#C49A00", "#FF9F6E",
    "#A3A500", "#39B600", "#00BF7D", "#00B0F6", "#9590FF", "#FF62BC",
    "#6C6C6C", "#D89000", "#FF6F00", "#D0BBFF", "#FFFF99", "#00FF00",
    "#00FFFF", "#FF00FF", "#1F3438", "#20B2AA"
]

color_map = dict(zip(variable_groups.keys(), ggplot_colors)) # create mapping from group to color
inv_map = {} # create inverse mapping from variables to groups

for v, k in no_dupes_vars.items():
  for l in k:
    inv_map[l] = v

In [ ]:
available = set(sum(variable_groups.values(), []))

In [ ]:
# iterate over matrix (lower left triangle)
# create connections list, including only absolute correlations >= threshold

connections_list = []

threshold = 0.25
columns = list(df.columns)

seen = set()

for i in range(0, df.shape[0]):
  for j in range(1, i + 1):
    value = df.iloc[i, j]
    row_name = columns[i + 1]
    col_name = columns[j]

    if row_name not in available or col_name not in available:
      continue

    if abs(value) >= threshold:
      # include
      from_color = color_map[inv_map[row_name]]
      to_color = color_map[inv_map[col_name]]
      connections_list += [(row_name, col_name, value, from_color, to_color)]
      seen.add(row_name)
      seen.add(col_name)

In [ ]:
kinds = type_map.keys()

# create hierarchy df
hierarchy_df = []

# initialize vertices array with origin
vertices = [("origin", "origin", "Residential Characteristics", "Objective Bio genes_neuroimaging")]

# add group vertices + edges first
for v in no_dupes_vars.keys():
  hierarchy_df += [("origin", v)]
  vertices += [(v, v, color_map[v], list(kinds)[0])]

# and then variables vertices + edges
for v, k in no_dupes_vars.items():
  hierarchy_df += [(v, l) for l in k]
  vertices += [(l, v, color_map[v], match_kind(l)) for l in k]

hierarchy_df

[('origin', 'Residential Characteristics'),
 ('origin', 'Ethnicity/Nationality'),
 ('origin', 'Diet/Nutrition'),
 ('origin', 'Physical Activity/Features'),
 ('origin', 'Technology Use'),
 ('origin', 'Religion'),
 ('origin', 'Family Dynamics & Parenting'),
 ('origin', 'Parent Social Functioning'),
 ('origin', 'Social Relationship Quality'),
 ('origin', 'School Dynamics'),
 ('origin', 'Adverse Life Events'),
 ('origin', 'Cognitive Task Outcomes'),
 ('origin', 'Sleep Problems'),
 ('origin', 'Medical/Somatic Problems'),
 ('origin', 'Externalizing'),
 ('origin', 'ADHD'),
 ('origin', 'Anxiety'),
 ('origin', 'Other Personality Features'),
 ('origin', 'Child Mood Issues'),
 ('origin', 'Child Delta'),
 ('origin', 'Parent Delta'),
 ('origin', 'Parent Mood Issues'),
 ('origin', 'Parent Anxiety'),
 ('origin', 'Parent Cognitive and Attention Issues'),
 ('origin', 'Parent Personality'),
 ('origin', 'Family Drug Use'),
 ('origin', 'SES & Mobility'),
 ('origin', 'Dynamic Cognitive Control Parameters')

In [ ]:
# saving dfs as csv

hierarchy_pandas_df = pd.DataFrame(hierarchy_df, columns=['from', 'to'])
hierarchy_pandas_df.to_csv('hierarchy_df.csv', index=False)

connections_data_frame = pd.DataFrame(connections_list, columns=['from', 'to', 'value', 'from_color', 'to_color'])
connections_data_frame.to_csv('connections_df.csv', index=False)

vertex_df = pd.DataFrame(vertices, columns=['name', 'group', 'color', 'type'])
vertex_df.to_csv('vertex_df.csv', index=False)